In [1]:
import openai
import os
import wikipedia

# 1.) Set up OpenAI and the enviornment


In [2]:
apikey = "#sk-dLJ7KKC4ExUyGY20fWHkT3BlbkFJ#aXFbx9dKHNvGFrbyFCBs#"
openai.api_key = apikey

In [3]:
client = openai.OpenAI(
    api_key = openai.api_key
)

In [4]:
dir(wikipedia)

['API_URL',
 'BeautifulSoup',
 'Decimal',
 'DisambiguationError',
 'HTTPTimeoutError',
 'ODD_ERROR_MESSAGE',
 'PageError',
 'RATE_LIMIT',
 'RATE_LIMIT_LAST_CALL',
 'RATE_LIMIT_MIN_WAIT',
 'RedirectError',
 'USER_AGENT',
 'WikipediaException',
 'WikipediaPage',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'cache',
 'datetime',
 'debug',
 'donate',
 'exceptions',
 'geosearch',
 'languages',
 'page',
 'random',
 're',
 'requests',
 'search',
 'set_lang',
 'set_rate_limiting',
 'set_user_agent',
 'stdout_encode',
 'suggest',
 'summary',
 'sys',
 'time',
 'timedelta',
 'unicode_literals',
 'util',
 'wikipedia']

# 2.) Use the wikipedia api to get a function that pulls in the text of a wikipedia page

In [5]:
# To loop through incorrect content in Wikipedia
page_titles = ["Artificial Intelligence", "Machine Learning"]
page_title = page_titles[0]

In [6]:
search_results = wikipedia.search(page_title)
search_results

['Artificial intelligence',
 'Generative artificial intelligence',
 'Artificial general intelligence',
 'A.I. Artificial Intelligence',
 'Applications of artificial intelligence',
 'Artificial intelligence art',
 'Hallucination (artificial intelligence)',
 'History of artificial intelligence',
 'Ethics of artificial intelligence',
 'Artificial intelligence in healthcare']

In [7]:
page = wikipedia.page(search_results[0])
page.content

'Artificial intelligence (AI) is the intelligence of machines or software, as opposed to the intelligence of living beings, primarily of humans. It is a field of study in computer science that develops and studies intelligent machines. Such machines may be called AIs.\nAI technology is widely used throughout industry, government, and science. Some high-profile applications are: advanced web search engines (e.g., Google Search), recommendation systems (used by YouTube, Amazon, and Netflix), interacting via human speech (such as Google Assistant, Siri, and Alexa), self-driving cars (e.g., Waymo), generative and creative tools (ChatGPT and AI art), and superhuman play and analysis in strategy games (such as chess and Go).Alan Turing was the first person to conduct substantial research in the field that he called machine intelligence. Artificial intelligence was founded as an academic discipline in 1956. The field went through multiple cycles of optimism, followed by periods of disappointm

In [8]:
def get_wikipedia_content(page_title):
    search_results = wikipedia.search(page_title)
    page = wikipedia.page(search_results[0])
    return(page.content)

# 3.) Build a chatgpt bot that will analyze the text given and try to locate any false info

In [9]:
wiki_content = get_wikipedia_content(page_title)

In [18]:
chat_completion  = client.chat.completions.create(
    model = "gpt-4", # Quote
    messages = [{"role" : "system", "content" : "I will be giving you an article let me know if anything is potentially false.  Go with afine tooth comb and have low sensitivity and locating potential errors. If there is errors please number them and be very concise when quoting them. If there is no potential errors found then just return 'NONE'"},
              {"role" : "user", "content" : wiki_content[:8192]}] # List
)
print(chat_completion.choices[0].message.content)

1. "AI technology is widely used throughout industry, government, and science. Some high-profile applications are: advanced web search engines (e.g., Google Search)..." - Potentially false. While Google does use AI in its search engines, the sentence could be misleading, as AI is just one of many tools used.

2. "Alan Turing was the first person to conduct substantial research in the field that he called machine intelligence." - Potentially false. While Alan Turing made significant contributions, it's not necessarily accurate to say he was the "first" to do substantial research.

3. "AI also draws upon psychology, linguistics, philosophy, neuroscience and other fields." - Potentially false. This statement is very broad, and while AI can be applied in these fields, it doesn't inherently draw upon them.

4. "Formal knowledge representations are used in content-based indexing and retrieval, scene interpretation, clinical decision support, knowledge discovery (mining "interesting" and acti

In [23]:
def chatgpt_error_correction(text):
    chat_completion  = client.chat.completions.create(
    model = "gpt-4", # Quote
    messages = [{"role" : "system", "content" : "I will be giving you an article let me know if anything is potentially false.  Go with afine tooth comb and have low sensitivity and locating potential errors. If there is errors please number them and be very concise when quoting them. If there is no potential errors found then just return 'NONE'"},
              {"role" : "user", "content" : text[:8192]}] # List
)
    return print(chat_completion.choices[0].message.content)

# 4.) Make a for loop and check a few wikipedia pages and return a report of any potentially false info via wikipedia

In [24]:
def split_text(text, chunk_size = 8180):
    chunks = len(text) // 8180 +  1
    return([text[i*chunk_size:(i+1)*chunk_size] for i in range(0, chunks - 1)])

In [20]:
split_text(text)

['The University of California, Los Angeles (UCLA) is a public land-grant research university in Los Angeles, California, United States. Its academic roots were established in 1881 as a normal school then known as the southern branch of the California State Normal School which later evolved into San José State University. The branch was transferred to the University of California, becoming the Southern Branch of UC in 1919, making it the second-oldest of the ten-campus University of California system after the University of California, Berkeley.\nUCLA offers 337 undergraduate and graduate degree programs in a range of disciplines, enrolling about 31,600 undergraduate and 14,300 graduate and professional students annually. It received 174,914 undergraduate applications for Fall 2022, including transfers, making it the most applied-to university in the United States. The university is organized into the College of Letters and Science and twelve professional schools. Six of the schools of

In [16]:
len(split_text(text))

7

In [25]:
page_titles = ["Artificial Intelligence", "Machine Learning", "UCLA"]
for page_title in page_titles:
    print("__________________" + page_title)
    try:
        text =  get_wikipedia_content(page_title)
        print(chatgpt_error_correction(text[:8192]))
    except:
        print("ERROR")

__________________Artificial Intelligence
1. "AI technology is widely used throughout industry, government, and science. Some high-profile applications are: advanced web search engines (e.g., Google Search)...": While AI technology is being used throughout various sectors, saying it's used in 'advanced web search engines' like Google Search may be slightly misleading. Google Search uses algorithms to provide search results, but this does not necessarily equate to AI technology in the full sense of the term.

2. "The field went through multiple cycles of optimism, followed by periods of disappointment and loss of funding, known as AI winter.": This framing oversimplifies the historical development of the AI field. AI 'winters' were not just periods of disappointment and loss of funding; they were also periods of self-criticism, reassessment, and divergence towards other computational approaches.

3. "AI researchers have adapted and integrated a wide range of problem-solving techniques, 